In [1]:
import library
from IPython.display import display, HTML
import pandas
import collections
import math
import collections
import re
import nltk
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import time
from nltk.corpus import wordnet

import requests
import urllib.parse

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [2]:
to_fix = set()
to_fix.add('http://skedit.info/_lifeta_gaston')
to_fix.add('http://www.solidarites-entreprises.org/_guidigan_yelognisseviassehodarling')
to_fix.add('http://www.zowasel.com/_oche_jerry')
to_fix.add('http://www.gracedivineagriculture.com_ediedikouakoungorangouengoranedi_abevalereabevalerebroualphonseericguieadamaalexisyannickmasseuclotairevenceslasguieadamaalexisyannickabevalere')
to_fix.add('https://yetulab.com_rubango_daniel')
to_fix.add('http://www.afconinvestments.com_nakitandwenakiryassewakilyanga_priscillasuubinoelbrian')
to_fix.add('http://www.qotto.net_gnonhouetraoretraore_alexoladikpomodibomodibo')
to_fix.add('http://berag-ziane.com_ziane_hafidhou')
to_fix.add('http://www.ministry%20of%20youth%20and%20sports.com_reen_hatem')
to_fix.add('https://web.facebook.com/davenbakery_ongmessom_blanche')
to_fix.add('http://renacabenin.org_akabassi_togbedjimedard')

# Filtres

In [3]:
df_filter = library.Filter().X
df_filter = df_filter[~df_filter.index.isin(to_fix)]
display(df_filter.head())
display(df_filter.describe(include = 'all'))

,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
key_main,,,,,,
http://seneyakaare.com_ndiaye_diariyata,sante,35-50,1,0.0,0.0,0.0
https://wb.semoa-group.com/_adjamagbo_edem,finance,25-34,4,0.0,5000.0,77000.0
https://talenteum.com/_nicolas_goldstein,technologie,35-50,100,400000.0,2.0,1401.5
http://chapwifi.bj/_adjassa_azimath,media,25-34,6,NaN,NaN,48000.0
https://www.facebook.com/TZenSn/_diop_sokhnalaye,agriculture,25-34,4,NaN,NaN,2500.0


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
count,3438,3438,3438.000000,2.072000e+03,2.394000e+03,3.438000e+03
unique,13,6,NaN,NaN,NaN,NaN
top,agriculture,25-34,NaN,NaN,NaN,NaN
freq,694,1833,NaN,NaN,NaN,NaN
mean,NaN,NaN,59.297557,2.526381e+05,1.173738e+06,3.134706e+06
std,NaN,NaN,1510.173542,3.823265e+06,4.114702e+07,1.537529e+08
min,NaN,NaN,0.000000,-1.500000e+04,-5.000000e+04,-1.700000e+04
25%,NaN,NaN,4.000000,0.000000e+00,5.670000e+00,1.250000e+03
50%,NaN,NaN,7.000000,1.213000e+03,5.000000e+03,9.150000e+03
75%,NaN,NaN,12.000000,1.823100e+04,3.000000e+04,4.500000e+04


## Par categorie

In [4]:
#all
df_filter_category_agriculture_mobilite = df_filter[df_filter['categorie'].isin(['agriculture','mobilite'])]
display(df_filter_category_agriculture_mobilite.head())
display(df_filter_category_agriculture_mobilite.groupby('categorie').describe(include = 'all'))

#agriculture
df_filter_category_agriculture  = df_filter[df_filter['categorie'] == 'agriculture']
display(df_filter_category_agriculture.head())
display(df_filter_category_agriculture.describe(include = 'all'))

#mobiilite
df_filter_category_mobilite  = df_filter[df_filter['categorie'] == 'mobilite']
display(df_filter_category_mobilite.head())
display(df_filter_category_mobilite.describe(include = 'all'))

#persona
persona = {'age_pers':df_filter['age_pers'] == '25-34', 
           'nbr_salarie':df_filter['nbr_salarie'].isin(range(0,10)),
           '1_ca': (df_filter['ca_2017']>0) | (df_filter['ca_2018']>0)|(df_filter['ca_2019']>0),
           'ca2019':(df_filter['ca_2019']>0),
           'ca': (df_filter['ca_2017']>0) & (df_filter['ca_2018']>0)&(df_filter['ca_2019']>0) }

df_filter_persona = df_filter[(persona['age_pers'])&(persona['nbr_salarie'])&(persona['ca'])]

display(df_filter_persona.head())
display(df_filter_persona.describe(include = 'all'))

subset = {'all':df_filter_category_agriculture_mobilite, 'mobilite': df_filter_category_mobilite, 'agriculture':df_filter_category_agriculture,'persona': df_filter_persona}


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
key_main,,,,,,
https://www.facebook.com/TZenSn/_diop_sokhnalaye,agriculture,25-34,4,NaN,NaN,2500.0
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,agriculture,25-34,3,-2.0,-1.0,-2.0
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,agriculture,25-34,7,200.0,450.0,3800.0
http://www.retpaci.ci_kouassi_ayagazalo,agriculture,35-50,6,NaN,NaN,2985.0
http://www.ecobuildersms.com%20%20%20%20%20//%20%20https://web.facebook.com/EcobuildersMadeinSenegal/_ndieguene_ndeyemarieaida,agriculture,Moins de 25 ans,4,NaN,7500.0,15000.0


age_pers                                                   ...  \
               count unique    top freq mean  std  min  25%  50%  75%  ...   
categorie                                                              ...   
agriculture      694      6  25-34  381  NaN  NaN  NaN  NaN  NaN  NaN  ...   
mobilite         153      6  25-34   85  NaN  NaN  NaN  NaN  NaN  NaN  ...   

            ca_2019                                                     \
             unique top freq          mean           std   min     25%   
categorie                                                                
agriculture     NaN NaN  NaN  3.454527e+05  4.399824e+06 -21.0  2150.0   
mobilite        NaN NaN  NaN  1.328215e+06  1.326554e+07   0.0   500.0   

                                               
                 50%         75%          max  
categorie                                      
agriculture  10500.0  43278.4675   92448000.0  
mobilite     13000.0  90000.0000  164000000.0  

[2 rows x 55 columns]

,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
key_main,,,,,,
https://www.facebook.com/TZenSn/_diop_sokhnalaye,agriculture,25-34,4,NaN,NaN,2500.0
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,agriculture,25-34,3,-2.0,-1.0,-2.0
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,agriculture,25-34,7,200.0,450.0,3800.0
http://www.retpaci.ci_kouassi_ayagazalo,agriculture,35-50,6,NaN,NaN,2985.0
http://www.ecobuildersms.com%20%20%20%20%20//%20%20https://web.facebook.com/EcobuildersMadeinSenegal/_ndieguene_ndeyemarieaida,agriculture,Moins de 25 ans,4,NaN,7500.0,15000.0


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
count,694,694,694.000000,4.340000e+02,5.020000e+02,6.940000e+02
unique,1,6,NaN,NaN,NaN,NaN
top,agriculture,25-34,NaN,NaN,NaN,NaN
freq,694,381,NaN,NaN,NaN,NaN
mean,NaN,NaN,58.600865,4.196418e+05,5.664958e+05,3.454527e+05
std,NaN,NaN,763.332447,5.890146e+06,6.967316e+06,4.399824e+06
min,NaN,NaN,0.000000,-5.000000e+00,-8.550000e+02,-2.100000e+01
25%,NaN,NaN,5.000000,0.000000e+00,5.280000e+02,2.150000e+03
50%,NaN,NaN,8.000000,2.950000e+03,5.932015e+03,1.050000e+04
75%,NaN,NaN,14.000000,1.975000e+04,3.100000e+04,4.327847e+04


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
key_main,,,,,,
https://sourcenets.com/_renson_renson,mobilite,25-34,8,2000.0,15000.0,170000.000
https://coliba.ci_kone_yaya,mobilite,35-50,6,0.0,50000.0,180000.000
http://www.misstaxighana.com_nyador_esenam,mobilite,35-50,3,NaN,24500.0,42420.000
http://mediapressafrica.com/_dounkengzele_bricedilane,mobilite,Moins de 25 ans,5,22.5,20.0,16.457
http://www.safiride.com_erhabor_ikponmwosa,mobilite,Over 50,16,NaN,0.0,0.000


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
count,153,153,153.000000,8.900000e+01,1.050000e+02,1.530000e+02
unique,1,6,NaN,NaN,NaN,NaN
top,mobilite,25-34,NaN,NaN,NaN,NaN
freq,153,85,NaN,NaN,NaN,NaN
mean,NaN,NaN,14.424837,1.817013e+05,1.680862e+05,1.328215e+06
std,NaN,NaN,49.691773,7.833327e+05,5.934678e+05,1.326554e+07
min,NaN,NaN,0.000000,0.000000e+00,-1.600000e+01,0.000000e+00
25%,NaN,NaN,4.000000,0.000000e+00,0.000000e+00,5.000000e+02
50%,NaN,NaN,6.000000,1.000000e+00,3.570000e+03,1.300000e+04
75%,NaN,NaN,11.000000,1.800000e+04,5.000000e+04,9.000000e+04


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
key_main,,,,,,
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,agriculture,25-34,7,200.0,450.0,3800.0
https://diarra-agrobusiness-sarl-13.webself.net_diarra_aissata,agriculture,25-34,5,1670.0,2280.0,3198.0
http://www.talentincubator.com_viderot_gnonnanharryjeerjr,education,25-34,4,18210.0,19150.0,16000.0
https://sourcenets.com/_renson_renson,mobilite,25-34,8,2000.0,15000.0,170000.0
http://Www.bouboulait.com_sangho_boubou,agriculture,25-34,9,7000.0,17500.0,23500.0


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019
count,320,320,320.000000,320.000000,320.000000,3.200000e+02
unique,13,1,NaN,NaN,NaN,NaN
top,agriculture,25-34,NaN,NaN,NaN,NaN
freq,67,320,NaN,NaN,NaN,NaN
mean,NaN,NaN,5.434375,21081.416238,29683.538376,5.559007e+05
std,NaN,NaN,2.133934,82405.358593,89780.987623,9.166538e+06
min,NaN,NaN,0.000000,1.000000,1.000000,3.000000e+00
25%,NaN,NaN,4.000000,1000.000000,1995.500000,4.990000e+03
50%,NaN,NaN,5.500000,4000.000000,7000.000000,1.300000e+04
75%,NaN,NaN,7.000000,11553.750000,22038.500000,4.000000e+04


In [5]:
subset['all'].index

Index(['https://www.facebook.com/TZenSn/_diop_sokhnalaye',
       'http://www.sunumbay-agritech.sn/_aw_mohamedelbechir',
       'http://www.Gh+entreprise.con_ngombesekou_godelyherlych',
       'http://www.retpaci.ci_kouassi_ayagazalo',
       'http://www.ecobuildersms.com%20%20%20%20%20//%20%20https://web.facebook.com/EcobuildersMadeinSenegal/_ndieguene_ndeyemarieaida',
       'https://diarra-agrobusiness-sarl-13.webself.net_diarra_aissata',
       'https://sourcenets.com/_renson_renson', 'https://coliba.ci_kone_yaya',
       'http://www.familygreencorp.org_eyidi_victormarceljunior',
       'http://cooperativedesouvriersdubatiment.e-monsite.com/http-lamaisondesartisans-e-monsite-com-pages/a.html_gueye_ousmane',
       ...
       'http://www.mpetchad.com_mmedjimtoingarmemadji_reoutou',
       'http://www.mpetchad.com_ndoubahidi_bosco',
       'http://www.cbdibaong.org_gbovidemlan_cocoucyrille',
       'http://www.cbdibaong.org_bedie_suzannezouvennisse',
       'http://www.cbdibaong.org_

# Selection de texte

In [6]:
text = library.Transformation().X
text = text[~text.index.isin(to_fix)]
subset = {'all':df_filter_category_agriculture_mobilite.join(text), 'mobilite': df_filter_category_mobilite.join(text), 'agriculture':df_filter_category_agriculture.join(text)}

In [7]:
#findna[findna.isna()]
len(set(df_filter_category_agriculture_mobilite.index).difference(text.index))

0

# MOdelisation

In [8]:
category_select = 'agriculture'

subset_df = subset[category_select]
display(subset_df.head())
display(subset_df.fillna('na').describe(include = 'all'))

kw = library.Keyword_extraction(subset_df, file = 'stopwords_agri.txt').X
kw_ = kw
display(kw.head())

data_samples = kw[['prez_struc_lemm', 'prez_produit_struc_lemm', 'prez_marche_struc_lemm', 'prez_zone_struc_lemm', 'prez_objectif_struc_lemm', 'prez_innovante_struc_lemm', 'prez_duplicable_struc_lemm', 'prez_durable_struc_lemm']].agg(' '.join, axis=1)
#l = [a.split() for a in data_samples.values.tolist()]
#n_features = len(set([item for sublist in l for item in sublist]))
n_features = 1000
n_top_words = 20
models, _ = library.Models(n_components = 15, n_top_words = n_top_words, n_features = n_features).run_models(data_samples)
components_agriculture = models['NMF_Frobenius']['components']

def reverse_lda(X, models = models):
    model = models['LDA']['model']
    features  =  model['feature_names']
    vectorizer = model['vectorizer']
    model['tf_vectorizer'].set_params(vocabulary =  model['feature_names']) 
    tf_x = vectorizer.fit_transform(X)
    predict = model.fit(LDA['tf']).transform(tf_x)
    max_topic = np.where(predict == max(predict[0]))[1][0]
    pdf = predict
    return max_topic

def reverse_nmf(X,distance , models = models):
    #X = []
    #X.append(data) 
    #X = data.tolist()
    model = models['NMF_{}'.format(distance)]['model']
    data = models['NMF_{}'.format(distance)]['tf']
    model.fit(X = data)
    models['NMF_{}'.format(distance)]['vectorizer'].set_params(vocabulary = models['NMF_{}'.format(distance)]['feature_names']) 
    new_data = models['NMF_{}'.format(distance)]['vectorizer'].fit_transform(X)
    predict = model.transform(new_data)
    max_topic = np.where(predict == max(predict[0]))[1][0]
    return predict

print(len(data_samples))
distance ='Frobenius'
predict = reverse_nmf(data_samples, distance ='Frobenius' )
results_fro = pandas.DataFrame(predict)
results_fro.columns = ['{}_topic_{}'.format(distance, key) for key in results_fro.keys()]
results_fro = results_fro.set_index(data_samples.index)

max_topic = pandas.Series([np.where(predict == max(record))[1][0] for record in predict])
max_topic = max_topic.to_frame().set_index(data_samples.index)
max_topic.columns = ['max_topic_frobenius'] 


#distance ='Kullback'
#results_kullback = pandas.DataFrame(reverse_nmf(data_samples, distance = 'Kullback'))
#results_kullback.columns = ['{}_topic_{}'.format(distance,  key) for key in results_kullback.keys()]

#results_kullback = results.set_index(data_samples.index)


header = ['categorie', 'age_pers', 'nbr_salarie', 'ca_2017', 'ca_2018', 'ca_2019','max_topic_frobenius','Frobenius_topic_0', 'Frobenius_topic_1', 'Frobenius_topic_2','Frobenius_topic_3', 'Frobenius_topic_4', 'Frobenius_topic_5','Frobenius_topic_6', 'Frobenius_topic_7', 'Frobenius_topic_8','Frobenius_topic_9', 'Frobenius_topic_10', 'Frobenius_topic_11','Frobenius_topic_12', 'Frobenius_topic_13', 'Frobenius_topic_14']
kw_stats = pandas.concat([kw, results_fro,max_topic], axis=1)[header]
kw_stats_ = kw_stats
display(kw_stats.groupby('max_topic_frobenius').count()[['categorie','age_pers']])

#kw_stats.apply(lambda x: get_max(x['Frobenius_topic_0'], x['Frobenius_topic_1'], x['Frobenius_topic_2'],
#       x['Frobenius_topic_3'], x['Frobenius_topic_4'], x['Frobenius_topic_5'],
#       x['Frobenius_topic_6'], x['Frobenius_topic_7'], x['Frobenius_topic_8'],
#       x['Frobenius_topic_9'], x['Frobenius_topic_10'], x['Frobenius_topic_11'],
#       x['Frobenius_topic_12'], x['Frobenius_topic_13'], x['Frobenius_topic_14'],
#       x['Frobenius_topic_15'], x['Frobenius_topic_16'], x['Frobenius_topic_17'],
#       x['Frobenius_topic_18'], x['Frobenius_topic_19']), axis=1)

,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,prez_objectif_struc,prez_innovante_struc,prez_duplicable_struc,prez_durable_struc
key_main,,,,,,,,,,,,,,
https://www.facebook.com/TZenSn/_diop_sokhnalaye,agriculture,25-34,4,NaN,NaN,2500.0,tzen is a senegalese company that specializes...,tzen markets its own brand of teainfusion for ...,we are targeting the west african market at th...,west africa senegal,in the medium term we plan to produce boxes o...,our product stands out thanks to the organic c...,it is duplicable in the sense that the raw mat...,as you know food is the new major sustainable ...
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,agriculture,25-34,3,-2.0,-1.0,-2.0,sunu mbay is a digital agricultural platform t...,sunu mbay is an innovative digital agricultura...,we are going to start our activity in the sene...,senegal river valley,today we are witnessing a revolution in agricu...,sunu mbay is an innovative digital agricultura...,we will be able to gradually integrate the dat...,having noted that in the senegal river valley ...
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,agriculture,25-34,7,200.0,450.0,3800.0,the republic of the congo now has a population...,we offer a delivery service of quality cassava...,the national and international,we are first working on the two major cities o...,the current national market is estimated at b...,an innovative solution is a home delivery serv...,we bring a stable price of the product which i...,our packaging is made with degradable elements
http://www.retpaci.ci_kouassi_ayagazalo,agriculture,35-50,6,NaN,NaN,2985.0,the ivorian company gazalo holding gh is a com...,sojasante is cooked soy flour rich in protein...,our products are sold in supermarkets shop ...,our structure is located in the berlin distric...,gazalo holdings expansion project consists of ...,there are several baby foods available and eve...,it can be duplicated insofar as we adopt a pol...,this solution makes french and african cities ...
http://www.ecobuildersms.com%20%20%20%20%20//%20%20https://web.facebook.com/EcobuildersMadeinSenegal/_ndieguene_ndeyemarieaida,agriculture,Moins de 25 ans,4,NaN,7500.0,15000.0,ems is a young and innovative company speciali...,our product is an economical ecological and in...,we work with agricultural cooperatives,we recover nonbiodegradable waste in large urb...,in years we want to build community sheds in...,innovation lies first of all in the sectors th...,certainly the farmers of déni biram ndao and m...,from an economic point of view the lack of sto...


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,prez_objectif_struc,prez_innovante_struc,prez_duplicable_struc,prez_durable_struc
count,694,694,694.000000,694,694,6.940000e+02,694,694,694,694,694,694,694,694
unique,1,6,NaN,206,275,NaN,694,692,691,629,691,694,691,694
top,agriculture,25-34,NaN,na,na,NaN,agro vie is a social enterprise and biotechnol...,,our business is designed to focus on the poor ...,vide,long terme,it is innovative because it allows poor women ...,the concept is low cost recycling based organ...,our green houses and farming processes will us...
freq,694,381,NaN,260,192,NaN,1,2,2,52,2,1,2,1
mean,NaN,NaN,58.600865,NaN,NaN,3.454527e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,763.332447,NaN,NaN,4.399824e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,-2.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,5.000000,NaN,NaN,2.150000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,8.000000,NaN,NaN,1.050000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,14.000000,NaN,NaN,4.327847e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/Volumes/GoogleDrive/My Drive/PROJET DATA/DATA/Transformed/stopwords_agri.txt


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,...,prez_duplicable_struc,prez_durable_struc,prez_struc_lemm,prez_produit_struc_lemm,prez_marche_struc_lemm,prez_zone_struc_lemm,prez_objectif_struc_lemm,prez_innovante_struc_lemm,prez_duplicable_struc_lemm,prez_durable_struc_lemm
key_main,,,,,,,,,,,,,,,,,,,,,
https://www.facebook.com/TZenSn/_diop_sokhnalaye,agriculture,25-34,4,NaN,NaN,2500.0,tzen is a senegalese company that specializes...,tzen markets its own brand of teainfusion for ...,we are targeting the west african market at th...,west africa senegal,...,it is duplicable in the sense that the raw mat...,as you know food is the new major sustainable ...,tzen wellbeing carry douxt brand tea make loca...,tzen market brand teainfusion wellbeing brand ...,market moment product without distinction gend...,,produce box sachet triple production also wish...,product stand organic component market moreove...,duplicable sense raw material almost free char...,food sustainable remark report government food...
http://www.sunumbay-agritech.sn/_aw_mohamedelbechir,agriculture,25-34,3,-2.0,-1.0,-2.0,sunu mbay is a digital agricultural platform t...,sunu mbay is an innovative digital agricultura...,we are going to start our activity in the sene...,senegal river valley,...,we will be able to gradually integrate the dat...,having noted that in the senegal river valley ...,sunu mbay digital agricultural platform accomp...,sunu mbay digital agricultural direct competit...,go river valley saint louis matam bakel open w...,river valley,witness revolution agriculture technique data ...,sunu mbay digital agricultural revolutionize f...,gradually integrate data deploy application de...,note river valley producer determine quantity ...
http://www.Gh+entreprise.con_ngombesekou_godelyherlych,agriculture,25-34,7,200.0,450.0,3800.0,the republic of the congo now has a population...,we offer a delivery service of quality cassava...,the national and international,we are first working on the two major cities o...,...,we bring a stable price of the product which i...,our packaging is made with degradable elements,nearly highly unequal distribution two city al...,delivery service cassava city brazzaville thus...,,first two city brazzaville tip capital city,current market estimate market within,home delivery service cassava throughout city ...,bring stable product stick cassava one,packaging make degradable element
http://www.retpaci.ci_kouassi_ayagazalo,agriculture,35-50,6,NaN,NaN,2985.0,the ivorian company gazalo holding gh is a com...,sojasante is cooked soy flour rich in protein...,our products are sold in supermarkets shop ...,our structure is located in the berlin distric...,...,it can be duplicated insofar as we adopt a pol...,this solution makes french and african cities ...,ivorian gazalo hold gh transform soya bean flo...,sojasante cook soy flour rich protein vitamin ...,product supermarket shop private chain store p...,locate berlin bingerville lagunes eighteen cote,gazalo holding consist production soybean corn...,several baby food even prescribe doctor birth ...,duplicate insofar adopt policy direct personal...,make city sustainable partnership interconnect...
http://www.ecobuildersms.com%20%20%20%20%20//%20%20https://web.facebook.com/EcobuildersMadeinSenegal/_ndieguene_ndeyemarieaida,agriculture,Moins de 25 ans,4,NaN,7500.0,15000.0,ems is a young and innovative company speciali...,our product is an economical ecological and in...,we work with agricultural cooperatives,we recover nonbiodegradable waste in large urb...,...,certainly the farmers of déni biram ndao and m...,from an economic point of view the lack of sto...,em young construction storage shed agricultura...,product economical ecological intelligent stor...,agricultural cooperative,recover nonbiodegradable waste urban build cou...,build community shed rural collection bin tire...,innovation first connect agriculture 

done in 0.000s.
Extracting tf-idf features for NMF...
done in 0.174s.
Extracting tf features for LDA...
done in 0.178s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.412s.

Topics in NMF model (Frobenius norm):
Topic #0: product market food local make production packaging produce natural raw material order flour supermarket brand range child create spice industry
Topic #1: farmer farm smallholder platform food access produce market connect chain harvest rural agriculture mobile technology scale crop service community online
Topic #2: fish aquaculture tilapia feed pond farm catfish fresh protein aquaponics production farming vegetable specie breed job insect meal meet water
Topic #3: agricultural producer application data platform service information digital agriculture mobile management market advice technology crop field farmer web bring actor
Topic #4: organic fertilizer soil vegetable compost chemical production crop he

,categorie,age_pers
max_topic_frobenius,,
0,113,113
1,70,70
2,34,34
3,58,58
4,37,37
5,24,24
6,50,50
7,21,21
8,170,170


In [9]:
category_select = 'mobilite'

subset_df = subset[category_select]
display(subset_df.head())
display(subset_df.fillna('na').describe(include = 'all'))

kw = library.Keyword_extraction(subset_df, file = 'stopwords_mobi.txt').X

display(kw.head())

data_samples = kw[['prez_struc_lemm', 'prez_produit_struc_lemm', 'prez_marche_struc_lemm', 'prez_zone_struc_lemm', 'prez_objectif_struc_lemm', 'prez_innovante_struc_lemm', 'prez_duplicable_struc_lemm', 'prez_durable_struc_lemm']].agg(' '.join, axis=1)

n_features = 1000
n_top_words = 20
models, _ = library.Models(n_components = 15, n_top_words = n_top_words, n_features = n_features).run_models(data_samples)
components_mobilite = models['NMF_Frobenius']['components']

def reverse_lda(X, models = models):
    model = models['LDA']['model']
    features  =  model['feature_names']
    vectorizer = model['vectorizer']
    model['tf_vectorizer'].set_params(vocabulary =  model['feature_names'])
    tf_x = vectorizer.fit_transform(X)
    predict = model.fit(LDA['tf']).transform(tf_x)
    max_topic = np.where(predict == max(predict[0]))[1][0]
    pdf = predict
    return max_topic

def reverse_nmf(X,distance , models = models):
    model = models['NMF_{}'.format(distance)]['model']
    data = models['NMF_{}'.format(distance)]['tf']
    model.fit(X = data)
    models['NMF_{}'.format(distance)]['vectorizer'].set_params(vocabulary = models['NMF_{}'.format(distance)]['feature_names']) 
    new_data = models['NMF_{}'.format(distance)]['vectorizer'].fit_transform(X)
    predict = model.transform(new_data)
    max_topic = np.where(predict == max(predict[0]))[1][0]
    return predict

print(len(data_samples))
distance ='Frobenius'
predict = reverse_nmf(data_samples, distance ='Frobenius')
results_fro = pandas.DataFrame(predict)
results_fro.columns = ['{}_topic_{}'.format(distance, key) for key in results_fro.keys()]
results_fro = results_fro.set_index(data_samples.index)

max_topic = pandas.Series([np.where(predict == max(record))[1][0] for record in predict])
max_topic = max_topic.to_frame().set_index(data_samples.index)
max_topic.columns = ['max_topic_frobenius']


#distance ='Kullback'
#results_kullback = pandas.DataFrame(reverse_nmf(data_samples, distance = 'Kullback'))
#results_kullback.columns = ['{}_topic_{}'.format(distance,  key) for key in results_kullback.keys()]

#results_kullback = results.set_index(data_samples.index)


header = ['categorie', 'age_pers', 'nbr_salarie', 'ca_2017', 'ca_2018', 'ca_2019','max_topic_frobenius',
       'Frobenius_topic_0', 'Frobenius_topic_1', 'Frobenius_topic_2',
       'Frobenius_topic_3', 'Frobenius_topic_4', 'Frobenius_topic_5',
       'Frobenius_topic_6', 'Frobenius_topic_7', 'Frobenius_topic_8',
       'Frobenius_topic_9', 'Frobenius_topic_10', 'Frobenius_topic_11',
       'Frobenius_topic_12', 'Frobenius_topic_13', 'Frobenius_topic_14']
kw_stats = pandas.concat([kw, results_fro,max_topic], axis=1)[header]
display(kw_stats.groupby('max_topic_frobenius').count()[['categorie','age_pers']])

#kw_stats.apply(lambda x: get_max(x['Frobenius_topic_0'], x['Frobenius_topic_1'], x['Frobenius_topic_2'],
#       x['Frobenius_topic_3'], x['Frobenius_topic_4'], x['Frobenius_topic_5'],
#       x['Frobenius_topic_6'], x['Frobenius_topic_7'], x['Frobenius_topic_8'],
#       x['Frobenius_topic_9'], x['Frobenius_topic_10'], x['Frobenius_topic_11'],
#       x['Frobenius_topic_12'], x['Frobenius_topic_13'], x['Frobenius_topic_14'],
#       x['Frobenius_topic_15'], x['Frobenius_topic_16'], x['Frobenius_topic_17'],
#       x['Frobenius_topic_18'], x['Frobenius_topic_19']), axis=1)

,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,prez_objectif_struc,prez_innovante_struc,prez_duplicable_struc,prez_durable_struc
key_main,,,,,,,,,,,,,,
https://sourcenets.com/_renson_renson,mobilite,25-34,8,2000.0,15000.0,170000.000,sourcenets is an innovation software house sys...,the main project name is parksoul which is a p...,who are we for municipalities and cities in n...,tanzania and africa in general,with the rapid increase in smartphone uses in ...,the convenience of booking a parking lot onlin...,we provide very lower cost solution people wil...,this solution is designed for both developing ...
https://coliba.ci_kone_yaya,mobilite,35-50,6,0.0,50000.0,180000.000,plastics per day this puts us in a leading pos...,with our mobile technology we have tackled the...,our market is based on all populations produci...,côte divoire abidjan and suburbs ghana accra,our growth objectives and mediumterm vision is...,with our mobile technology we have tackled the...,we opted for low tech to hack plastic waste in...,our solution will make french and african citi...
http://www.misstaxighana.com_nyador_esenam,mobilite,35-50,3,NaN,24500.0,42420.000,miss taxi ghana is a local ghanaian registered...,our strategies and theory of change miss taxi ...,community public of accra for our transport se...,accra the capital of ghana is very cosmopolita...,strategic direction vision and goals within th...,the transport provision sector since the intro...,there is a never ending demand for mobility so...,training of women to participate in mobility s...
http://mediapressafrica.com/_dounkengzele_bricedilane,mobilite,Moins de 25 ans,5,22.5,20.0,16.457,média press africa was born out of the desire ...,insecurity is a major problem affecting most m...,the fight against crime is our hobbyhorse so i...,our main target is africa as a whole but for t...,the viamap application was a resounding succes...,the community aspect of our platform is its ma...,our solution requires minimal investment costs...,the development of a city rhymes with safety h...
http://www.safiride.com_erhabor_ikponmwosa,mobilite,Over 50,16,NaN,0.0,0.000,as a ridesharing green transport system and de...,safi offers innovative platformbased transport...,safi is basically for everyone as long as you ...,the first point of deployment of safiride in r...,safis growth objective is to provide an etrans...,safi emotor is equipped with one of a kind fea...,we believe that having successfully piloted sa...,the growing population and increasing urbaniza...


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,prez_objectif_struc,prez_innovante_struc,prez_duplicable_struc,prez_durable_struc
count,153,153,153.000000,153,153,1.530000e+02,153,153,153,153,153,153,153,153
unique,1,6,NaN,41,61,NaN,153,153,153,140,153,153,153,153
top,mobilite,25-34,NaN,na,na,NaN,technovera is a social impact organisation tha...,the trotro diaries community is our current pr...,egyptian market,vide,our longterm target market is to reach all of ...,we have a radical approach of directly sourcin...,we only need to develop once and can be deploy...,niochi makes people feel at home all over afri...
freq,153,85,NaN,64,48,NaN,1,1,1,11,1,1,1,1
mean,NaN,NaN,14.424837,NaN,NaN,1.328215e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,49.691773,NaN,NaN,1.326554e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,4.000000,NaN,NaN,5.000000e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,6.000000,NaN,NaN,1.300000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,11.000000,NaN,NaN,9.000000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/Volumes/GoogleDrive/My Drive/PROJET DATA/DATA/Transformed/stopwords_mobi.txt


,categorie,age_pers,nbr_salarie,ca_2017,ca_2018,ca_2019,prez_struc,prez_produit_struc,prez_marche_struc,prez_zone_struc,...,prez_duplicable_struc,prez_durable_struc,prez_struc_lemm,prez_produit_struc_lemm,prez_marche_struc_lemm,prez_zone_struc_lemm,prez_objectif_struc_lemm,prez_innovante_struc_lemm,prez_duplicable_struc_lemm,prez_durable_struc_lemm
key_main,,,,,,,,,,,,,,,,,,,,,
https://sourcenets.com/_renson_renson,mobilite,25-34,8,2000.0,15000.0,170000.000,sourcenets is an innovation software house sys...,the main project name is parksoul which is a p...,who are we for municipalities and cities in n...,tanzania and africa in general,...,we provide very lower cost solution people wil...,this solution is designed for both developing ...,sourcenets innovation software house system in...,name parksoul park management system devise st...,municipality city genuine revenue collection o...,,rapid smartphone us internet penetration see p...,convenience book parking online save estimate ...,pay normal parking fee get commission park owner,design develop city smart city therefore well
https://coliba.ci_kone_yaya,mobilite,35-50,6,0.0,50000.0,180000.000,plastics per day this puts us in a leading pos...,with our mobile technology we have tackled the...,our market is based on all populations produci...,côte divoire abidjan and suburbs ghana accra,...,we opted for low tech to hack plastic waste in...,our solution will make french and african citi...,plastic per put us leading amount plastic coll...,mobile technology tackle plastic waste collect...,market produce waste home every organisation c...,suburbs accra,mediumterm double pellet production subject do...,mobile technology tackle plastic waste collect...,opt tech hack plastic waste last therefore app...,make city sustainable environmental social sen...
http://www.misstaxighana.com_nyador_esenam,mobilite,35-50,3,NaN,24500.0,42420.000,miss taxi ghana is a local ghanaian registered...,our strategies and theory of change miss taxi ...,community public of accra for our transport se...,accra the capital of ghana is very cosmopolita...,...,there is a never ending demand for mobility so...,training of women to participate in mobility s...,miss taxi local ghanaian register notforprofit...,strategy theory miss taxi partnership local de...,community public accra transport service logis...,accra capital cosmopolitan diverse mobility wo...,strategic direction goal within next five miss...,transport since introduction automobile tradit...,never end mobility whatever shape form woman d...,training woman mobility city go get woman dome...
http://mediapressafrica.com/_dounkengzele_bricedilane,mobilite,Moins de 25 ans,5,22.5,20.0,16.457,média press africa was born out of the desire ...,insecurity is a major problem affecting most m...,the fight against crime is our hobbyhorse so i...,our main target is africa as a whole but for t...,...,our solution requires minimal investment costs...,the development of a city rhymes with safety h...,média press bear information stakeholder privi...,insecurity affect city particular indeed demog...,crime hobbyhorse order maximum model choose gi...,whole moment platform already deploy conquer g...,viamap application resounding success first re...,community aspect platform asset collaborative ...,require minimal investment implementation depl...,city rhyme safety health infrastructure one pi...
http://www.safiride.com_erhabor_ikponmwosa,mobilite,Over 50,16,NaN,0.0,0.000,as a ridesharing green transport system and de...,safi offers innovative platformbased transport...,safi is basically for everyone as long as you ...,the first point of deployment of safiride in r...,...,we believe that having successfully piloted sa...,the growing population and increasing urbaniza...,ridesharing green transport system delivery se...,safi platformbased transportation strategy acc...,safi basically everyone move access affordable...,first deployment safiride city city capital lo...,safis et

done in 0.000s.
Extracting tf-idf features for NMF...
done in 0.044s.
Extracting tf features for LDA...
done in 0.036s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 0.065s.

Topics in NMF model (Frobenius norm):
Topic #0: transport bus ticket public payment city buy travel mode traveller ride vehicle card purchase application reservation digital passenger possibility cooperative
Topic #1: plastic waste material recycle brick collection building pave stone household raw construction recycling product housing collect recycled job bin sort
Topic #2: electric vehicle power charge station energy solar charging battery emission petrol fuel driver fossil transportation mobility owner market production motion
Topic #3: city service application mobile technology make local road information mobility platform data urban digital market community tourist management access develop
Topic #4: carpooling travel passenger driver trip save ap

,categorie,age_pers
max_topic_frobenius,,
0,11,11
1,9,9
2,8,8
3,72,72
4,6,6
5,4,4
6,4,4
7,6,6
8,5,5


In [10]:
target_agriculture = {'Topic 0': 'product market food local make production packaging produce natural raw material moringa order flour supermarket create brand child range spice'
,
'Topic 1': 'vide young finance field fertilizer female feed fee farming farmer farm family fair factory facilitate extract extend export exploitation exploit'
,
'Topic 2': 'farmer farm smallholder platform access food produce market connect rural chain agriculture harvest mobile technology scale crop service community data'
,
'Topic 3': 'fish aquaculture tilapia feed farm catfish pond protein aquaponics production fresh farming vegetable specie insect breed water meal job meet'
,
'Topic 4': 'organic fertilizer soil mushroom vegetable compost chemical production crop healthy health plant produce farmer garden food manure agriculture pesticide eat'
,
'Topic 5': 'agricultural producer application data platform service information digital agriculture mobile management market advice technology crop web field production bring farmer'
,
'Topic 6': 'cassava flour tuber machine crop yam production energy state maize staple pest pollution food security duplicate wheat social common transform'
,
'Topic 7': 'feed meat poultry animal livestock chicken farm egg pig protein pork breed fee production broiler breeding hen chick produce breeder'
,
'Topic 8': 'city woman urban food community agriculture rural production vegetable market young sustainable farm social promote make local land create school'
,
'Topic 9': 'honey beekeeping hive bee shea forest tree village reforestation production pure local income product biodiversity butter conservation deforestation train participatory'
,
'Topic 10':'water irrigation pump drinking treatment treat management technology solar tank sanitation resource plant innovation engineering maintenance source save application current'
,
'Topic 11': 'fruit juice pineapple natural vegetable mango jam bottle production ginger tree dry dried transform orange restaurant sugar bar fresh raw'
,
'Topic 12': 'waste plastic compost material recycle create manufacture energy environmental fertilizer sustainable feed household protein environment transform management source city insect'
,
'Topic 13': 'oil palm soap cosmetic coconut essential production factory brand cake range skin flower treatment ginger flour nut market industry river'
,
'Topic 14': 'coffee export bean grower tea sustainability sustainable produce banana impact material culture package buy nut brand transformation source packaging share'
}
target_mobilite = {'Topic 0': 'city service application mobility road mobile technology digital market data platform make information urban taxi traffic community driver management transport'
,
'Topic 1': 'plastic waste material recycle collection brick salinity building pave household stone raw construction recycling housing product collect recycled job bin'
,
'Topic 2': 'electric vehicle power charge station energy solar charging battery driver emission petrol fuel fossil mobility owner market production transportation motion'
,
'Topic 3': 'transport public payment ride mode vehicle city travel route data operator build card electronic frenchspeaking journey cooperative traveller urban wallet'
,
'Topic 4': 'vide young export expert exchange event evaluation estimate estate establish essential erp equipment equip equality environmentally environmental environment entry entrepreneurship'
,
'Topic 5': 'carpooling travel driver passenger trip save application money vehicle traveller connect geographical individual digital interurban carpool leisure airport share employee'
,
'Topic 6': 'park parking search operator city municipality pay revenue agent payment management municipal vehicle resident commuter registration make motorist smarter app'
,
'Topic 7': 'delivery parcel ecommerce relay food courier logistics dhl urban digitize vehicle supplier restaurant distribution ecosystem market carrier cover transport actor'
,
'Topic 8': 'light street solar traffic energy renewable city rural production road intelligent automation association public safety case equipment panel intervention government'
,
'Topic 9': 'estate real image ai record search digital architecture stop platform house save room shop home efficient satisfy strategic marketing regard'
,
'Topic 10': 'transportation parent school child service driver route age logistics daily state pillar motorcycle class specific safety worker transporter app government'
,
'Topic 11': 'bicycle bike community rider repair university service mobility shop daily tyre app power friendly security trip smarter student platform environmentally'
,
'Topic 12': 'commuter matatu pay psv fare card wallet smart matatus cashless public track operator operation crew mobile daily owner money travel'
,
'Topic 13': 'bus ticket passenger transport service buy card transportation investment gps traveller mobility purchase possibility productivity commute urban metro money capital'
,
'Topic 14': 'tourist tourism local site guide cultural organization accommodation sustainable ecotourism discover visit collaboration ecological promote make information count tatas visitor'
}

In [11]:
import numpy
numpy.std(kw_stats[kw_stats['max_topic_frobenius'] ==0].values.tolist()[1][7:])
#numpy.std([0,0,0,0,0,0])
t = [1/15]*15
t[0] = 1/16
numpy.std(t)

0.0010393492741038722

In [12]:
[max(e[7:]) for e in kw_stats[kw_stats['max_topic_frobenius'] ==0].values.tolist()]#[0][7:]

[0.11256261422960716,
 0.3439243674426471,
 0.324429854730846,
 0.3245839912150939,
 0.1894212965578714,
 0.29563169282590224,
 0.3772686636434623,
 0.460256903928649,
 0.14480578815951733,
 0.35367948724388115,
 0.3901991647792066]

In [13]:
to_consider = {}
to_consider['mobilite'] = [3,5,7,8,12]
to_consider['agriculture'] = [2,4,5,8,9,10,12]

In [14]:
map_topic = dict()

topic_match = dict()
for topic in components_agriculture:
    for target in target_agriculture:
        topic_match['c_t_{}_{}'.format(topic, target)] = sum([1 for element in components_agriculture[topic] if element in target_agriculture[target]])

def findtopicx(x):
    list_prop = []
    for t in ['c_t_{}_Topic {}'.format(topic,x) for topic in components_agriculture]:
        list_prop.append(topic_match[t])
    print('topicx',x)
    print(['{}%'.format(round(i*100/n_top_words)) for i in list_prop], 'max:', max([round(i*100/n_top_words) for i in list_prop]))
    return list_prop.index(max(list_prop))
print('agriculture')
map_topic['agriculture'] = {'Topic {}'.format(x): findtopicx(x) for x in to_consider['agriculture']}

topic_match = dict()
for topic in components_mobilite:
    for target in target_mobilite:
        topic_match['c_t_{}_{}'.format(topic, target)] = sum([1 for element in components_mobilite[topic] if element in target_mobilite[target]])

def findtopicx(x):
    list_prop = []
    for t in ['c_t_{}_Topic {}'.format(topic,x) for topic in components_mobilite]:
        list_prop.append(topic_match[t])
    print('topicx',x)
    print(['{}%'.format(round(i*100/n_top_words)) for i in list_prop], 'max:', max([round(i*100/n_top_words) for i in list_prop]))
    return list_prop.index(max(list_prop))
print('\nMobilite')

map_topic['mobilite'] = {'Topic {}'.format(x): findtopicx(x) for x in to_consider['mobilite']}

agriculture
topicx 2
['25%', '70%', '5%', '15%', '30%', '5%', '10%', '0%', '10%', '10%', '15%', '0%', '15%', '0%', '10%'] max: 70
topicx 4
['25%', '20%', '10%', '15%', '60%', '10%', '5%', '10%', '0%', '35%', '10%', '5%', '30%', '15%', '20%'] max: 60
topicx 5
['30%', '65%', '10%', '25%', '25%', '5%', '20%', '0%', '0%', '25%', '20%', '5%', '25%', '5%', '25%'] max: 65
topicx 8
['45%', '15%', '20%', '30%', '30%', '10%', '10%', '0%', '55%', '15%', '10%', '20%', '25%', '10%', '10%'] max: 55
topicx 9
['15%', '0%', '5%', '0%', '5%', '5%', '5%', '0%', '20%', '35%', '5%', '10%', '15%', '5%', '5%'] max: 35
topicx 10
['0%', '10%', '10%', '15%', '0%', '0%', '0%', '5%', '0%', '0%', '35%', '0%', '10%', '0%', '0%'] max: 35
topicx 12
['10%', '0%', '30%', '25%', '25%', '5%', '10%', '5%', '10%', '10%', '10%', '30%', '0%', '0%', '0%'] max: 30

Mobilite
topicx 3
['35%', '5%', '5%', '0%', '20%', '30%', '5%', '0%', '10%', '0%', '20%', '10%', '5%', '5%', '5%'] max: 35
topicx 5
['25%', '0%', '5%', '10%', '55%'

In [15]:
missing = {'http://www.colina.com_sodegla_colin', 'https://vc4a.com/members/sydwell-mcebo/_sihlangu_sydwellmcebo', 'http://www.cbdibaong.org_gbovidemlan_cocoucyrille', 'http://www.mpetchad.com_idriss_abakarmahamat', 'http://blueandgreeneng.com_achimi_ziadath', 'http://WWW.GROUPFJM.COM_machel_fernando', 'http://www.clavisatlas.net_hema_jeanpaul', 'http://www.inexence.group_viallon_laurent', 'http://www.cbdibaong.org_bigo_sedjrogwladysbertille', 'http://www.ebikes4africa.org_walther_marita', 'http://www.centreagriinnov.com_bangourabangourabarry_bangalybangalybinta', 'http://www.cbdibaong.org_aholou_pulcherie', 'http://www.cbdibaong.org_akokponsode_hippolyte', 'http://www.mpetchad.com_mmedjimtoingarmemadji_reoutou', 'http://www.goudirysurlenet.tv_sakho_hamet', 'https://seekewa.com_zamble_frederic', 'http://www.elvirabeninong.org_michael_capochichi', 'http://www.trimarkaquaculturecentre.com_yeboahagyepongsey_marksuzettenanaadjoaemefa', 'http://www.dattiers-Niger.com_sandasarouhou_mahamadoulaouali', 'http://www.solidarites-entreprises.org/_fakambi_kemi', 'https://www.facebook.com/Yeimyorgnicfruits/?modal=admin_todo_tour_kpemissi_solim', 'http://www.cbdibaong.org_houedanou_martin', 'http://www.agrona.net_elkafafy_mohamad', 'http://www.bbc-drc.com_okomaamboheki_aubinmerveille', 'http://www.me.bf/newkieta_zabsonre_abdoulaziz', 'https://web.facebook.com/Full-Development-Agency-1989721877958767/_ntakobajira_louange', 'http://www.cbdibaong.org_yemadje_emeline', 'http://www.ypard.net_rubonekakashemwa_landry', 'http://www.cbdibaong.org_tchegninougbo_marcelline', 'http://www.opastogo.org_komi_maglo', 'http://blolab.org_agbayazon_cossimedard', 'http://www.dronetechafrfica.com_mahamat_issaabakar', 'http://www.cbdibaong.org_bedie_suzannezouvennisse', 'https://horticulture.ucdavis.edu/drycard_odette_bahati', 'http://www.mpetchad.com_ndoubahidi_bosco'}

In [16]:
mising_agriculture = df_filter[(df_filter.index.isin(missing)) & (df_filter['categorie'] == 'agriculture')].index
missing_mobilite = df_filter[(df_filter.index.isin(missing)) & (df_filter['categorie'] == 'mobilite')].index

In [17]:
for element in missing:
    print(element, element in set(library.Filter().X.index))

http://www.cbdibaong.org_tchegninougbo_marcelline True
http://blueandgreeneng.com_achimi_ziadath True
https://www.facebook.com/Yeimyorgnicfruits/?modal=admin_todo_tour_kpemissi_solim True
http://www.clavisatlas.net_hema_jeanpaul True
https://web.facebook.com/Full-Development-Agency-1989721877958767/_ntakobajira_louange True
http://www.dattiers-Niger.com_sandasarouhou_mahamadoulaouali True
http://www.goudirysurlenet.tv_sakho_hamet True
http://www.solidarites-entreprises.org/_fakambi_kemi True
http://www.inexence.group_viallon_laurent True
http://www.ypard.net_rubonekakashemwa_landry True
http://www.mpetchad.com_ndoubahidi_bosco True
http://www.me.bf/newkieta_zabsonre_abdoulaziz True
https://horticulture.ucdavis.edu/drycard_odette_bahati True
http://www.cbdibaong.org_akokponsode_hippolyte True
http://www.cbdibaong.org_bigo_sedjrogwladysbertille True
http://www.mpetchad.com_idriss_abakarmahamat True
http://www.cbdibaong.org_yemadje_emeline True
http://www.mpetchad.com_mmedjimtoingarmemadj

In [18]:
for element in missing:
    print(element, element in set(library.Transformation().X.index))

http://www.cbdibaong.org_tchegninougbo_marcelline True
http://blueandgreeneng.com_achimi_ziadath True
https://www.facebook.com/Yeimyorgnicfruits/?modal=admin_todo_tour_kpemissi_solim True
http://www.clavisatlas.net_hema_jeanpaul True
https://web.facebook.com/Full-Development-Agency-1989721877958767/_ntakobajira_louange True
http://www.dattiers-Niger.com_sandasarouhou_mahamadoulaouali True
http://www.goudirysurlenet.tv_sakho_hamet True
http://www.solidarites-entreprises.org/_fakambi_kemi True
http://www.inexence.group_viallon_laurent True
http://www.ypard.net_rubonekakashemwa_landry True
http://www.mpetchad.com_ndoubahidi_bosco True
http://www.me.bf/newkieta_zabsonre_abdoulaziz True
https://horticulture.ucdavis.edu/drycard_odette_bahati True
http://www.cbdibaong.org_akokponsode_hippolyte True
http://www.cbdibaong.org_bigo_sedjrogwladysbertille True
http://www.mpetchad.com_idriss_abakarmahamat True
http://www.cbdibaong.org_yemadje_emeline True
http://www.mpetchad.com_mmedjimtoingarmemadj

In [19]:
for element in mising_agriculture:
    print(element, element in set(kw_.index))

http://www.solidarites-entreprises.org/_fakambi_kemi True
http://www.bbc-drc.com_okomaamboheki_aubinmerveille True
http://www.inexence.group_viallon_laurent True
https://horticulture.ucdavis.edu/drycard_odette_bahati True
http://www.dattiers-Niger.com_sandasarouhou_mahamadoulaouali True
http://www.opastogo.org_komi_maglo True
http://www.dronetechafrfica.com_mahamat_issaabakar True
http://www.goudirysurlenet.tv_sakho_hamet True
http://www.colina.com_sodegla_colin True
http://www.trimarkaquaculturecentre.com_yeboahagyepongsey_marksuzettenanaadjoaemefa True
https://seekewa.com_zamble_frederic True
https://www.facebook.com/Yeimyorgnicfruits/?modal=admin_todo_tour_kpemissi_solim True
http://www.clavisatlas.net_hema_jeanpaul True
https://vc4a.com/members/sydwell-mcebo/_sihlangu_sydwellmcebo True
https://web.facebook.com/Full-Development-Agency-1989721877958767/_ntakobajira_louange True
http://WWW.GROUPFJM.COM_machel_fernando True
http://www.elvirabeninong.org_michael_capochichi True
http://b

In [20]:
for element in missing_mobilite:
    print(element, element in set(kw.index))

http://blolab.org_agbayazon_cossimedard True
http://www.ebikes4africa.org_walther_marita True
http://www.me.bf/newkieta_zabsonre_abdoulaziz True


In [21]:
for element in mising_agriculture:
    print(element, element in set(kw_stats_.index))

http://www.solidarites-entreprises.org/_fakambi_kemi True
http://www.bbc-drc.com_okomaamboheki_aubinmerveille True
http://www.inexence.group_viallon_laurent True
https://horticulture.ucdavis.edu/drycard_odette_bahati True
http://www.dattiers-Niger.com_sandasarouhou_mahamadoulaouali True
http://www.opastogo.org_komi_maglo True
http://www.dronetechafrfica.com_mahamat_issaabakar True
http://www.goudirysurlenet.tv_sakho_hamet True
http://www.colina.com_sodegla_colin True
http://www.trimarkaquaculturecentre.com_yeboahagyepongsey_marksuzettenanaadjoaemefa True
https://seekewa.com_zamble_frederic True
https://www.facebook.com/Yeimyorgnicfruits/?modal=admin_todo_tour_kpemissi_solim True
http://www.clavisatlas.net_hema_jeanpaul True
https://vc4a.com/members/sydwell-mcebo/_sihlangu_sydwellmcebo True
https://web.facebook.com/Full-Development-Agency-1989721877958767/_ntakobajira_louange True
http://WWW.GROUPFJM.COM_machel_fernando True
http://www.elvirabeninong.org_michael_capochichi True
http://b

In [22]:
for element in missing_mobilite:
    print(element, element in set(kw_stats.index))

http://blolab.org_agbayazon_cossimedard True
http://www.ebikes4africa.org_walther_marita True
http://www.me.bf/newkieta_zabsonre_abdoulaziz True
